#  Differentiable Optimization with cuProx + PyTorch

This notebook demonstrates **GPU-accelerated differentiable optimization layers** using cuProx.

We'll show:
1. How to embed QP solvers in neural networks (OptNet-style)
2. Backpropagation through optimization
3. **Performance comparison** vs scipy
4. Training a network with optimization layers


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import time
import sys
sys.path.insert(0, '../python')

# Set style for beautiful plots
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'monospace'

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


## 1️⃣ Basic Differentiable QP

Solve a simple QP and compute gradients through the solution:


In [ ]:
from cuprox.torch import QPLayer, solve_qp

# Create a simple QP: min (1/2)||x||^2 - q'x
# Analytical solution: x* = q

n = 5
P = torch.eye(n, dtype=torch.float64)
q = torch.tensor([1., 2., 3., 4., 5.], dtype=torch.float64, requires_grad=True)

# Solve
x = solve_qp(P, -q)

print(" Solution x* =", x.detach().numpy())
print("   Expected  q =", q.detach().numpy())

# Compute gradients
loss = x.sum()
loss.backward()

print("\n Gradient ∂loss/∂q =", q.grad.numpy())
print("   (Shows how changing q affects the optimal x*)")


## 2️⃣  Benchmark: cuProx vs scipy

Compare solve time for QP problems of different sizes:


In [ ]:
from scipy.optimize import minimize

def benchmark_cuprox(n_vars, n_problems=50):
    """Benchmark cuProx QP solver with gradients."""
    times = []
    for _ in range(n_problems):
        P = torch.eye(n_vars, dtype=torch.float64)
        q = torch.randn(n_vars, dtype=torch.float64, requires_grad=True)
        
        start = time.time()
        x = solve_qp(P, q)
        x.sum().backward()
        times.append(time.time() - start)
    return np.mean(times) * 1000, np.std(times) * 1000

def benchmark_scipy(n_vars, n_problems=50):
    """Benchmark scipy BFGS solver."""
    times = []
    for _ in range(n_problems):
        P = np.eye(n_vars)
        q = np.random.randn(n_vars)
        
        start = time.time()
        minimize(lambda x: 0.5 * x @ P @ x + q @ x, x0=np.zeros(n_vars), method='BFGS')
        times.append(time.time() - start)
    return np.mean(times) * 1000, np.std(times) * 1000

# Run benchmarks
sizes = [5, 10, 20, 50, 100, 200]
cuprox_results = []
scipy_results = []

print(" Running benchmarks...")
for n in sizes:
    cp = benchmark_cuprox(n)
    sp = benchmark_scipy(n)
    cuprox_results.append(cp[0])
    scipy_results.append(sp[0])
    print(f"   n={n:3d}: cuProx={cp[0]:6.2f}ms, scipy={sp[0]:6.2f}ms, speedup={sp[0]/cp[0]:.1f}x")

print("\n Benchmarks complete!")


In [ ]:
# Visualize benchmark results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Time comparison
ax = axes[0]
x_pos = np.arange(len(sizes))
width = 0.35

bars1 = ax.bar(x_pos - width/2, cuprox_results, width, label='cuProx', color='#00ff88', edgecolor='white')
bars2 = ax.bar(x_pos + width/2, scipy_results, width, label='scipy', color='#ff6b6b', edgecolor='white')

ax.set_xlabel('Problem Size (variables)', fontsize=12, fontweight='bold')
ax.set_ylabel('Time (ms)', fontsize=12, fontweight='bold')
ax.set_title('⚡ Solve Time: cuProx vs scipy', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(sizes)
ax.legend(loc='upper left', fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Speedup chart
ax = axes[1]
speedup = np.array(scipy_results) / np.array(cuprox_results)
colors = ['#00ff88' if s >= 1 else '#ff6b6b' for s in speedup]

bars = ax.bar(sizes, speedup, color=colors, edgecolor='white', linewidth=2)
ax.axhline(y=1, color='yellow', linestyle='--', alpha=0.7, linewidth=2, label='Break-even')
ax.set_xlabel('Problem Size', fontsize=12, fontweight='bold')
ax.set_ylabel('Speedup Factor', fontsize=12, fontweight='bold')
ax.set_title(' cuProx Speedup (with gradients)', fontsize=14, fontweight='bold')

for bar, s in zip(bars, speedup):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
            f'{s:.1f}x', ha='center', fontsize=11, fontweight='bold', color='white')

ax.grid(True, alpha=0.3, axis='y')
ax.legend()

plt.tight_layout()
plt.savefig('pytorch_benchmark.png', dpi=150, bbox_inches='tight', facecolor='#1a1a2e')
plt.show()

print(f"\n Average speedup: {np.mean(speedup):.1f}x faster than scipy (including gradient computation)!")


## 📝 Summary

### cuProx PyTorch Integration:

| Feature | cuProx | Others |
|---------|--------|--------|
| **GPU Acceleration** | ✅ Native CUDA | ❌ CPU only |
| **Implicit Differentiation** | ✅ Efficient | ⚠️ Varies |
| **API Simplicity** | ✅ `solve_qp(P, q)` | ⚠️ Verbose |
| **PyTorch Native** | ✅ Full autograd | ✅ |

### Quick Start:
```python
from cuprox.torch import QPLayer, solve_qp

# Functional API
x = solve_qp(P, q, lb=lb, ub=ub)
x.sum().backward()  # Gradients work!

# Module API (for nn.Sequential)
layer = QPLayer(n_vars=10)
x = layer(P, q)
```
